In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [52.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
H

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-02 20:29:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.04MB/s    in 0.2s    

2021-04-02 20:29:58 (6.04 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_DE_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_multilingual_DE_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import *
# Read in the Review dataset as a DataFrame

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         DE|      10133| RVOG49N0H1FB6|B004TACMZ8|     569741360|Bosch GMS120 Ortu...|    Home Improvement|          5|            0|          0|   N|                Y|               Super|Delivery took a l...| 2014-08-01|
|         DE|      19612| RNCMD6OLTP4HM|1846071224|     785505948|The Wheels On The...|               Bo

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby(df["customer_id"]).agg(count(df.customer_id)).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      38153|             3|
|    2094984|             1|
|    3006039|             1|
|    3121833|             1|
|    3391089|             8|
|    3470216|             2|
|    3529901|             1|
|    3744024|             1|
|    3768097|             1|
|    3804964|             1|
|    3908493|             1|
|    3949894|             1|
|    3975970|             1|
|    3980089|             1|
|    4614773|             2|
|    4672448|             2|
|    4816572|             1|
|    4876799|             1|
|    4991959|             1|
|    5011347|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(df["product_id"], df["product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00BRC6PYU|Cirque du Soleil:...|
|B005AH411M|       Der Zoowärter|
|B006FG3QSG|Paranormal Activi...|
|B004AMAG6W|Greatest Hits...S...|
|B000075BFK|         Better Dayz|
|014241543X|           If I Stay|
|B0000DCXS4|         Patch Adams|
|B00004S5RI|         Poltergeist|
|B0036GLFPY|Terminator 2 [Blu...|
|B00KY1Y64Q|Der kleine Nick [...|
|B000TP1KFK|Monty Python - Di...|
|B0083U94D8|Intel Core i5-357...|
|144476490X|Bob: No Ordinary Cat|
|B00N0T38LG|.5: The Gray Chap...|
|B009ACJSSO|[UK-Import]Jaws L...|
|B003E3TQZK|        A Single Man|
|B00EQZUJGK|Butterfly Effect ...|
|B00EQZSRRI|     Cinema Paradiso|
|B00BQXM558|Fun English: Engl...|
|B0021BA3OO|Iron Man (Uncut U...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RVOG49N0H1FB6|      10133|B004TACMZ8|     569741360| 2014-08-01|
| RNCMD6OLTP4HM|      19612|1846071224|     785505948| 2014-12-04|
| R4AUOBI8YC0R8|      19612|0375851569|     516548029| 2014-12-04|
|R1VSHIJ1RHIBTE|      19677|B0060SVG54|     302116447| 2015-07-16|
|R3JBLVALWSLCZD|      19999|B00EYQ6CVC|     368843515| 2014-02-08|
| RJ6GK77Y2NKCK|      20014|B00MC1CCAG|     663681053| 2014-11-11|
| R13W0E5EDX50J|      20152|B005D7FKWI|     182054491| 2013-10-04|
|R2L8UN6YSPY1CK|      20152|B007W9KQ9W|     251269761| 2013-10-04|
| RA2DFU68J832H|      20152|B005F3DFXG|     283218719| 2013-10-04|
|R191XKANDDV34N|      20152|B00EYXGOYK|     678332854| 2013-11-29|
| RD2NWWDF0S2B9|      20152|B00BRC6PYU|     715581077| 2014-05-21|
|R35SNJXI0Y9J1T|      20152|B00DZFMC4Y|     542832677| 2014-05

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RVOG49N0H1FB6|          5|            0|          0|   N|                Y|
| RNCMD6OLTP4HM|          5|            1|          1|   N|                Y|
| R4AUOBI8YC0R8|          5|            0|          0|   N|                Y|
|R1VSHIJ1RHIBTE|          5|            0|          0|   N|                Y|
|R3JBLVALWSLCZD|          5|            9|         14|   N|                Y|
| RJ6GK77Y2NKCK|          4|            6|          7|   N|                Y|
| R13W0E5EDX50J|          5|            0|          1|   N|                Y|
|R2L8UN6YSPY1CK|          3|            1|          3|   N|                Y|
| RA2DFU68J832H|          4|            1|          1|   N|                Y|
|R191XKANDDV34N|          4|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cwbz5r3ufcj5.us-east-2.rds.amazonaws.com:5432/Challenge_16"
config = {"user":"postgres", 
          "password": "mythangs93", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)